In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
train = pd.read_csv('/kaggle/input/melting-points/train.csv')
test  = pd.read_csv('/kaggle/input/melting-points/test.csv')

In [ ]:
print('train shape', train.shape)
print('test shape', test.shape)
print(train.head())

In [ ]:
drop_cols = ['id', 'SMILES']
X = train.drop(columns=drop_cols + ['Tm'])
y = train['Tm']   # check units: this dataset says Kelvin
X_test = test.drop(columns=drop_cols)

X = X.fillna(X.median())
X_test = X_test.fillna(X_test.median())

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=200, max_depth=12, random_state=42, n_jobs=-1)
model.fit(X_tr, y_tr)

val_preds = model.predict(X_val)
print('Validation MAE:', mean_absolute_error(y_val, val_preds))

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_mae = -cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error', n_jobs=-1)
print('5-fold CV MAE mean:', cv_mae.mean(), 'std:', cv_mae.std())

model.fit(X, y)
test_preds = model.predict(X_test)

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'Tm': test_preds})
submission.to_csv('submission.csv', index=False)
print('submission.csv created')
print(submission.head())

train shape (2662, 427)
test shape (666, 426)
     id                       SMILES      Tm  Group 1  Group 2  Group 3  \
0  2175        FC1=C(F)C(F)(F)C1(F)F  213.15        0        0        0   
1  1222  c1ccc2c(c1)ccc3Nc4ccccc4c23  407.15        0        0        0   
2  2994          CCN1C(C)=Nc2ccccc12  324.15        2        1        0   
3  1704                   CC#CC(=O)O  351.15        1        0        0   
4  2526                    CCCCC(S)C  126.15        2        3        0   

   Group 4  Group 5  Group 6  Group 7  ...  Group 415  Group 416  Group 417  \
0        0        0        0        0  ...          0          0          0   
1        0        0        0        0  ...          0          0          0   
2        0        0        0        0  ...          0          0          0   
3        0        0        0        0  ...          0          0          0   
4        0        0        0        0  ...          0          0          0   

   Group 418  Group 419  Gro